In [54]:
import pandas as pd
import os
import csv
from collections import defaultdict

BASE_DIR = "data_Jan2021/"
RETWATCH_DATA = BASE_DIR+"retwatch_hum_retractions.csv"
p_retwatch = pd.read_csv(RETWATCH_DATA)

#Normalize series

# --- Column 1
column = "Retraction or Other Notices\r\nTitle/Subject(s)/Journal --- Publisher/Affiliation(s)/Retraction Watch Post URL(s)\r\n490 Item(s) Found"
p_retwatch[column] = p_retwatch[column].apply(lambda x : x.split("\r\n"))
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"title":x[0],"subject":x[1],"venue":x[2],"other":{"value":x[3:] if len(x) > 3 else None,"description":"Affiliation(s)/Retraction Watch Post URL(s)"}})
# --- Column 2
column = "Reason(s)"
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"value": [a.lower().strip() for a in list(filter(None,x.split("+")))] })
# --- Column 3
column = "Author(s)"
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"value": x.split("\r\n")})
# --- Column 4
column = "Original Paper\r\nDate/PubMedID/DOI"
p_retwatch[column] = p_retwatch[column].apply(lambda x : x.split("\r\n"))
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"date":x[0],"id":x[1:] if len(x) > 1 else [""]})
# --- Column 5
column = "Retraction or Other Notices\r\nDate/PubMedID/DOI"
p_retwatch[column] = p_retwatch[column].apply(lambda x : x.split("\r\n"))
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"date":x[0],"id":x[1:] if len(x) > 1 else [""]})
# --- Column 6
column = "Article Type(s)\r\nNature of Notice"
p_retwatch[column] = p_retwatch[column].apply(lambda x : x.split("\r\n"))
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"type":x[0],"ret_notice":x[1:]})
# --- Column 7
column = "Countries\r\nPaywalled?\r\nNotes"
p_retwatch[column] = p_retwatch[column].apply(lambda x : x.split("\r\n"))
p_retwatch[column] = p_retwatch[column].apply(lambda x : {"country":x[0],"paywall":x[1] if len(x) > 1 else None,"other": {"value": x[2:] if len(x) > 2 else [""], "description":"additional notes"}})

# Create a csv containing all the retracted items

In [62]:
l_csv_items = []
index = set()
for row in p_retwatch.values:
    pubmed_id = ""
    doi_id = ""
    pub_date = ""
    
    pub_date = row[4]["date"]
    if len(row[4]["id"]) > 0:
        an_id = row[4]["id"][0]
        if an_id != '' and an_id != 'unavailable' and an_id != '00000000':
            pubmed_id = an_id
        if len(row[4]["id"]) > 1:
            an_id = row[4]["id"][1]
            if an_id != '' and an_id != 'unavailable' and an_id != '00000000':
                doi_id = an_id

    key = (row[1]["title"],doi_id,pub_date)
    if key not in index and "Retraction" in row[6]["ret_notice"]:
        l_csv_items.append({
            "title": row[1]["title"],
            "doi": doi_id,
            "pubmedid": pubmed_id,
            "date": row[4]["date"],
            "authors": "; ".join(row[3]["value"]),
            "subject": row[1]["subject"],
            "venue": row[1]["venue"],
            "article_type": row[6]["type"],
            "country": row[7]["country"],
            "paywall": row[7]["paywall"],
            "notes": "; ".join(row[7]["other"]["value"]),
            "other_article_info": "; ".join(row[1]["other"]["value"]),
            "retraction_reasons": "; ".join(row[2]["value"]),
            "retraction_date": row[5]["date"],
            "retraction_notice_type": "; ".join(row[6]["ret_notice"]),
            "retraction_notice_doi": "; ".join(row[5]["id"]),
        })
        index.add(key)
    else:
        for r in l_csv_items:
            if (r["title"],r["doi"],r["date"]) == key:
                r["retraction_reasons"] = "; ".join(row[2]["value"]) + ";; "+ r["retraction_reasons"]
                r["retraction_date"] = row[5]["date"] + ";; " + r["retraction_date"]
                r["retraction_notice_type"] = "; ".join(row[6]["ret_notice"]) + ";; "+ r["retraction_notice_type"]
                r["retraction_notice_doi"] = "; ".join(row[5]["id"]) + ";; "+ r["retraction_notice_doi"]
                
#if '' or 'unavailable' or '00000000'
df_items = pd.DataFrame(l_csv_items)
df_items.to_csv('retracted_items_1.csv',index=False)